In [38]:
from time import time
import numpy as np
with open('19.txt', 'r') as f:
    data = f.read().splitlines()

In [39]:
prices = []
for line in data:
    line = [int(x) for x in line.replace(':', '.').split() if x.isdigit()]
    prices.append(line)

resources = {
    'ore': 0,
    'clay': 0,
    'obsidian': 0,
    'geode': 0
}

robots = {
    'ore': 1,
    'clay': 0,
    'obsidian': 0,
    'geode': 0,
}

In [33]:
from copy import deepcopy


class Factory:
    def __init__(self, prices):
        self.prices = prices
        self.max_ore = max(prices[0], prices[1], prices[2], prices[4])
        self.max_clay = prices[3]
        self.max_obsidian = prices[5]
        
    def find_possible_item(self, recources, robots, time):
        possible_items = []
        
        ore = recources['ore']
        obsidian = recources['obsidian']
        clay = recources['clay']
        
        robots_ore = robots['ore']
        robots_clay = robots['clay']
        robots_obsidian = robots['obsidian']
        
        if (self.prices[4] <= ore) and (self.prices[5] <= obsidian):
            possible_items.append('geode')
        if (self.prices[0] <= ore) and (robots_ore * time + ore < self.max_ore * time):
            possible_items.append('ore')
        if (self.prices[1] <= ore) and (robots_clay * time + clay < self.max_clay * time):
            possible_items.append('clay')
        
        if ((self.prices[2] <= ore) and \
            (self.prices[3] <= clay) and \
            (robots_obsidian * time + obsidian < time * self.max_obsidian)):
            possible_items.append('obsidian')
        
        
        possible_items.append('skip')
        return possible_items

    def buy_robot(self, resources, robots, robot):
        new_resources = deepcopy(resources)
        new_robots = deepcopy(robots)
        if robot != 'skip':
            new_robots[robot] += 1
        if robot == 'ore':
            new_resources['ore'] -= self.prices[0]
            return new_resources, new_robots
        if robot == 'clay':
            new_resources['ore'] -= self.prices[1]
            return new_resources, new_robots
        if robot == 'obsidian':
            new_resources['ore'] -= self.prices[2]
            new_resources['clay'] -= self.prices[3]
            return new_resources, new_robots
        if robot == 'geode':
            new_resources['ore'] -= self.prices[4]
            new_resources['obsidian'] -= self.prices[5]
            return new_resources, new_robots
        return new_resources, new_robots

    def find_max_geo(self, time, robots, resources, max_geode):
#         print(time, robots, resources, max_geode)
        if time == 0:
            return resources['geode']
        geode = resources['geode']
        
        
        
        if geode + np.sum(robots['geode'] +  np.arange(time)) <= max_geode: # geode + time*(time+1) + robots['geode'] but time -= 1 before.
            return 0
        time -=1
        
        
        if robots['ore'] >= self.prices[4] and robots['obsidian'] >= self.prices[5]:
            return geode + np.sum(robots['geode'] +  np.arange(time))
        else:
            for act in self.find_possible_item(resources, robots, time):
                new_resources, new_robots = self.buy_robot(resources, robots, act)
                for k, v in robots.items():
                    new_resources[k] += v
                max_geode = max(max_geode, self.find_max_geo(time, new_robots, new_resources, max_geode))
            return max_geode
        return -1

In [47]:
results = []
for i, price in enumerate(prices):
    f1 = Factory(price)
    max_geo = f1.find_max_geo(24, robots, resources, 0)
    print(i+1, max_geo)
    results.append((i+1, max_geo))

2 1
3 2
4 6
5 0
6 13
7 6
8 3
9 1
10 2
11 7
12 0
13 2
14 0
15 1
16 1
17 2
18 0
19 0
20 0
21 0
22 0
23 6
24 3
25 1
26 3
27 1
28 1
29 5
30 4


In [53]:
results_2 = []
for i, price in enumerate(prices[:3]):
    f1 = Factory(price)
    max_geo = f1.find_max_geo(32, robots, resources, 0)
    print(i+1, max_geo)
    results_2.append((i+1, max_geo))

1 32
2 21
3 28


In [55]:
np.prod([x[1] for x in results_2])

18816